In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf


In [7]:
image_path = "/cat_image.jpg"

img = cv2.imread(image_path)
img = cv2.resize(img, (256, 256))

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

input_tensor = gray.reshape(1, 256, 256, 1).astype(np.float32) / 255.0

In [15]:
vertical_kernel = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)

kernel_tensor = vertical_kernel.reshape((3, 3, 1, 1))
input_shape = (256, 256, 1)
